# 评估模型

上面的测试过程是一张一张图片进行预测的，这样的话速度会很慢，所以我们可以对图片进行批量处理，这样的话速度会快很多。所以我们需要对预测函数进行修改，使其能够批量处理图片。

### 定义测试集

In [1]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((256, 256))
])

# 将灰度图的数据集转换成RGB图像
transform_gray = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((256, 256))
])

class TestDataset(torch.utils.data.Dataset):
    def __init__(
            self, 
            data_dir, 
            img_paths,
            transform=transform,
            transform_gray=transform_gray
        ):
        self.data_dir = data_dir
        self.transform = transform
        self.transform_gray = transform_gray
        # self.imgs = os.listdir(data_dir)
        self.imgs = img_paths

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.imgs[idx])
        img = Image.open(img_path)
        if img.mode == 'RGB':
            img = self.transform(img)
        elif img.mode == 'L':
            img = self.transform_gray(img)
        return img

### 查看提交格式

提前查看一下测试集的信息

In [2]:
import pandas as pd
test_images = pd.read_csv('test_images.csv', header=0)
test_images.head()

,image_id,image_path,width,height
0,45331,0045331.jpg,640,169
1,35639,0035639.jpg,640,176
2,33770,0033770.jpg,640,178
3,42479,0042479.jpg,640,186
4,44556,0044556.jpg,640,193


In [3]:
my_submit = pd.DataFrame(columns=['image_id', 'predictions'])
my_submit['image_id'] = test_images['image_id']

In [4]:
img_paths = test_images['image_path'].tolist()
img_paths[:5]

['0045331.jpg', '0035639.jpg', '0033770.jpg', '0042479.jpg', '0044556.jpg']

In [5]:
test_data_dir = "./dl_detection/test/"
Cocodata_test = TestDataset(
    test_data_dir,
    img_paths
)
Cocodata_test.imgs[0], Cocodata_test.__len__()

('0045331.jpg', 16362)

## 开始批量预测

加载训练好的模型参数

In [6]:
from utils.device import try_gpu
from mydet.model import R50_FPN_SSD
device = try_gpu()
# 在这里加载训练好的模型
net = torch.load('resnet50_fpn_ssd_huawei2.pth',map_location=device)

In [7]:
batch_size = 128
test_dataloader = torch.utils.data.DataLoader(
    Cocodata_test,
    batch_size=batch_size,
    shuffle=False
)

定义预测函数

In [8]:
from mydet.bbox.postprocessor import BBoxPostProcessor
from mydet.bbox.postprocessor import bbox_filter
import torch.nn.functional as F
from utils.submit import bboxListToStr

bbpp = BBoxPostProcessor(
    nms_threshold=0.5,
    neg_threshold=0.00999,
    device = device
)

def predict_batch(X, batch_idx):
    net.eval()
    with torch.no_grad():
        batch_size = X.shape[0]
        anchors, cls_preds, bbox_preds = net(X.to(device))
        cls_probs = F.softmax(cls_preds, dim=2).permute(0, 2, 1) # 利用softmax转换成概率
        output = bbpp.multibox_detection(cls_probs, bbox_preds, anchors)
        # 去除掉背景类
        for sample_idx in range(batch_size):
            # 获取图片的宽和高信息
            image_idx = batch_idx * batch_size + sample_idx
            w = test_images['width'][image_idx]
            h = test_images['height'][image_idx]
            bbox_scale = torch.tensor([w, h, w, h], dtype=torch.float32, device=device)
            # 去除掉背景类
            idx = [i for i, sample in enumerate(output[sample_idx]) if sample[0] != -1]
            sample_without_background = output[sample_idx][idx]
            # 过滤掉一些预测框
            sample_good = bbox_filter(sample_without_background)
            # 如果一个类别的预测框超过300个，只取前300个（置信度从高到低排序）
            if len(sample_good) > 300:
                sample_good = sample_good[:300]

            bbox_xyxy = sample_good[:, 2:] * bbox_scale
            bbox_labels = sample_good[:, 0]
            bbox_confs = sample_good[:, 1]

            sample_good = torch.cat(
                [
                    bbox_xyxy,
                    bbox_confs.unsqueeze(1),
                    bbox_labels.unsqueeze(1),
                ],
                dim=1
            )

            # 将预测框转换成字符串并保存到my_submit中
            my_submit.loc[image_idx, 'predictions'] = bboxListToStr(sample_good.cpu().numpy().tolist())
            # print('batch_idx: %d, sample_idx: %d, sample_good: %d' % (batch_idx, sample_idx, len(sample_good)))


In [9]:
from utils.print_tools import print_progress_bar
# 开始预测
for batch_idx, X in enumerate(test_dataloader):
    # 如果my_submit.csv已经存在该部分数据的预测结果，则跳过
    if batch_idx * batch_size < my_submit['predictions'].count() :
        continue

    predict_batch(X, batch_idx)
    
    print_progress_bar(
        batch_idx + 1, 
        len(test_dataloader), 
        prefix='Progress:', 
        suffix='Complete', 
        decimals=2,
        length=50,
        fill = "█"
    )
    # 每5个batch保存一次
    if (batch_idx + 3) % 3 == 0:
        my_submit.to_csv('my_submit.csv', index=False)

KeyboardInterrupt: 

In [ ]:
# def predict(X):
#     net.eval()
#     with torch.no_grad():
#         batch_size = X.shape[0]
#         anchors, cls_preds, bbox_preds = net(X.to(device))
#         cls_probs = F.softmax(cls_preds, dim=2).permute(0, 2, 1) # 利用softmax转换成概率
#         output = bbpp.multibox_detection(cls_probs, bbox_preds, anchors)
#         return output

In [ ]:
# from utils.print_tools import print_progress_bar
# all_predictions = torch.tensor([], dtype=torch.float32, device=device) # 保存所有的预测结果
# for batch_idx, X in enumerate(test_dataloader):
#     output = predict(X)
#     all_predictions = torch.cat([all_predictions, output], dim=0)
#     print_progress_bar(
#         batch_idx + 1, 
#         len(test_dataloader), 
#         prefix='Progress:', 
#         suffix='Complete', 
#         decimals=2,
#         length=50,
#         fill = "█"
#     )

In [10]:
my_submit.head()

,image_id,predictions
0,45331,{2.332752227783203 1.1542993783950806 2.333015...
1,35639,{8.562759399414062 2.1747426986694336 8.564172...
2,33770,{2.200969934463501 0.9442633390426636 2.201041...
3,42479,{73.32339477539062 51.60209655761719 234.37561...
4,44556,{162.06549072265625 0.0720682144165039 162.065...


In [11]:
my_submit.to_csv('my_submit.csv', index=False)